# BMEN Final Project

Install dependencies

In [2]:
!python3 -m pip install -r requirements.txt

In [7]:
!python3 -m pip install nibabel

  Using cached nibabel-5.3.2-py3-none-any.whl.metadata (9.1 kB)
Using cached nibabel-5.3.2-py3-none-any.whl (3.3 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# Find which movies include anxiety information 

In [43]:
import os
import pandas as pd
from glob import glob

base_path = "/Users/emmasombers/BMENfinalproject/ds004892"
event_files = glob(os.path.join(base_path, "sub-*", "ses-*", "beh", "*_events.tsv"))

files_with_anxiety = []
files_without_anxiety = []

for file in event_files:
    try:
        df = pd.read_csv(file, sep="\t")
        if "Anxiety" in df.columns:
            files_with_anxiety.append(file)
        else:
            files_without_anxiety.append(file)
    except Exception as e:
        print(f"Error reading {file}: {e}")

print(f"\nFound {len(files_with_anxiety)} files WITH 'Anxiety' column")
print(f"Found {len(files_without_anxiety)} files WITHOUT 'Anxiety' column")

# Save full lists to file if you want:
with open("files_with_anxiety.txt", "w") as out:
    out.write("\n".join(files_with_anxiety))


Found 43 files WITH 'Anxiety' column
Found 371 files WITHOUT 'Anxiety' column


# Extract subject, movie, anxiety, onset, duration from each anxiety file and create json with this info 

In [45]:
from collections import defaultdict
import os
import pandas as pd
import json

subject_movie_events = defaultdict(list)

for file in files_with_anxiety:
    try:
        df = pd.read_csv(file, sep="\t")
        filename = os.path.basename(file)

        # Extract subject and task/movie
        subject = filename.split("_")[0].replace("sub-", "")
        task = [x for x in filename.split("_") if "task-" in x][0].replace("task-", "")

        for _, row in df.iterrows():
            if pd.notna(row.get("Anxiety")):
                subject_movie_events[(subject, task)].append({
                    "anxiety": row["Anxiety"],
                    "onset": row["onset"],
                    "duration": row["duration"]
                })
    except Exception as e:
        print(f"Skipping {file}: {e}")


# Convert tuple keys to strings to make it JSON serializable
serializable_data = {
    f"{subject}::{movie}": events
    for (subject, movie), events in subject_movie_events.items()
}

with open("subject_movie_events.json", "w") as f:
    json.dump(serializable_data, f, indent=2)

# Download the preprocessed, denoised fMRI data that matches the files with anxiety scores

In [5]:
# Create a list of paths of the fMRI files we need to download 
import os 
fmri_paths = []

with open("files_with_anxiety.txt", "r") as f:
    for line in f:
        line = line.strip()
        if line == "":
            continue

        # Extract parts from the path
        parts = line.split("/")
        sub = next(p for p in parts if p.startswith("sub-"))
        ses = next(p for p in parts if p.startswith("ses-"))
        fname = os.path.basename(line)

        # Extract task name
        task = [x for x in fname.split("_") if "task-" in x][0].replace("task-", "")

        # Build corresponding fMRI filename
        fmri_path = f"ds004892/derivatives/preprocessing/{sub}/{ses}/func/{sub}_{ses}_task-{task}_space-MNI_desc-ppres_bold.nii.gz"
        fmri_paths.append(fmri_path)

# Print or write to a file
print("\nfMRI files to download:")
for path in fmri_paths:
    print(path)

# write to shell script for datalad
with open("get_fmri_files.sh", "w") as f:
    for path in fmri_paths:
        f.write(f"datalad get {path}\n")


🧠 fMRI files to download:
ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-Superhero_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-Sintel_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-TearsOfSteel_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-BetweenViewings_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-ToClaireFromSonny_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-Chatter_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-Spaceman_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocessing/sub-S01/ses-2/func/sub-S01_ses-2_task-AfterTheRain_space-MNI_desc-ppres_bold.nii.gz
ds004892/derivatives/preprocess

# Create new folder only with the relevant fMRI files 

In [ ]:
import os
import shutil

count = 0 

# Load paths from your shell script
with open("get_fmri_files.sh", "r") as f:
    fmri_paths = [line.strip().split("datalad get ")[-1] for line in f if line.startswith("datalad get")]

# Destination directory
subset_dir = "subset_fmri"
os.makedirs(subset_dir, exist_ok=True)

# Move files 
for path in fmri_paths:
    if os.path.exists(path):
        dest = os.path.join(subset_dir, os.path.basename(path))
        shutil.move(path, dest)
        print(f"✅ Moved: {path}")
    else:
        print(f"⚠️ Skipped (not found): {path}")
    count +=1
    print(count)
    

⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-Superhero_space-MNI_desc-ppres_bold.nii.gz
1
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-Sintel_space-MNI_desc-ppres_bold.nii.gz
2
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-4/func/sub-S01_ses-4_task-TearsOfSteel_space-MNI_desc-ppres_bold.nii.gz
3
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-BetweenViewings_space-MNI_desc-ppres_bold.nii.gz
4
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-ToClaireFromSonny_space-MNI_desc-ppres_bold.nii.gz
5
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-Chatter_space-MNI_desc-ppres_bold.nii.gz
6
⚠️ Skipped (not found): ds004892/derivatives/preprocessing/sub-S01/ses-3/func/sub-S01_ses-3_task-Spaceman_space-MNI_desc-ppres_bold.nii.